In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 
import os

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Dropout, LeakyReLU, Flatten,  MaxPool2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split

from tqdm import tqdm, tqdm_notebook


2025-08-18 20:50:49.965610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755550249.978491       9 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755550249.982437       9 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 20:50:49.997705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.layers.pooling'

In [3]:
train_dir = '../input/train/train/'
test_dir = '../input/test/test/'


In [4]:
train_data = pd.read_csv('../input/train.csv')
train_data.head()


,id,has_cactus
0,2de8f189f1dce439766637e75df0ee27.jpg,1
1,36704d250f236238e7f996812c48235d.jpg,1
2,eacde22fdc8c175972a5768e3daa8bc9.jpg,1
3,5d442f834da5e57d22b24802c32a8ca8.jpg,1
4,152491e0daf75c0e669400300ff7e645.jpg,1


In [5]:
features = []
labels = []

images = train_data['id'].values

for img_id in tqdm_notebook(images):
    features.append(cv2.imread(train_dir+img_id))
    labels.append(train_data[train_data['id'] == img_id]
                  ['has_cactus'].values[0])
    
features = np.asarray(features)
features = features.astype('float32')
features /= 255
labels = np.asarray(labels)


NameError: name 'tqdm_notebook' is not defined

In [6]:
# Data augmentation
datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center= False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=10,  #randomly rotate image in 0 to 180 degree
            zoom_range=0.1, #randomly zoom image
            width_shift_range=0.1, #randomly shift images horizontally
            height_shift_range=0.1, #randomly shift images vertically
            horizontal_flip = False, #randomly flip images
            vertical_flip = False #randomly flip images
)
datagen.fit(features)


NameError: name 'ImageDataGenerator' is not defined

In [7]:
# split dataset
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size =0.1, random_state=2)


NameError: name 'train_test_split' is not defined

In [8]:
# define model
model = Sequential()
model.add(Conv2D(15, kernel_size=3, activation='relu',
                 input_shape=(32,32,3), padding='same'))
model.add(Conv2D(15, kernel_size=3, activation='relu',
                 padding='same'))
model.add(Conv2D(15, kernel_size=3, activation='relu',
                 padding='same'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-18 20:50:55.338937: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 15)     │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 15)     │         2,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 15)     │         2,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15360)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        15,361 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,861 (77.58 KB)

 Trainable params: 19,861 (77.58 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [10]:
#set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 15
batch_size = 86


NameError: name 'ReduceLROnPlateau' is not defined

In [11]:
clf = model.fit_generator(datagen.flow(X_train, y_train,batch_size=batch_size), 
                          epochs=epochs, validation_data=(X_val, y_val), verbose=2,
                          steps_per_epoch=X_train.shape[0] // batch_size, 
                          callbacks=[learning_rate_reduction])


AttributeError: 'Sequential' object has no attribute 'fit_generator'

In [12]:
test_features = []
test_images = []

for img_id in tqdm_notebook(os.listdir(test_dir)):
    test_features.append(cv2.imread(test_dir+img_id))
    test_images.append(img_id)
    
test_features = np.asarray(test_features)
test_features = test_features.astype('float32')
test_features /= 255


NameError: name 'tqdm_notebook' is not defined

In [13]:
model.save('cactus_model.h5')


In [14]:
# Running the model over the test images

test_predictions = model.predict(test_features)
submissions = pd.DataFrame(test_predictions, columns=['has_cactus'])
submissions['has_cactus'] = submissions['has_cactus'].apply(lambda x: 1 if x > 0.75 else 0)
submissions['id'] = ''
cols = submissions.columns.tolist()
cols = cols[-1:] + cols[:-1]
submissions=submissions[cols]


KeyError: 'pop from an empty set'

In [15]:
for i, img in enumerate(test_images):
    submissions.set_value(i,'id',img)


In [16]:
# Saving the output file

submissions.to_csv('submission.csv',index=False)


NameError: name 'submissions' is not defined